In [47]:
%gui qt
%matplotlib qt
import xmcd_projection
from xmcd_projection import deep_reload
deep_reload(xmcd_projection)
from xmcd_projection import *
from glob import glob
import os

In [5]:
folder_path = r"D:\Magnumfe\al6-1-1_peem_profile2_10mT15deg"

data_folder = os.path.join(folder_path, "data")
mag_files = glob(os.path.join(data_folder, "*.csv"))
msh_file = glob(os.path.join(data_folder, "*.msh"))[0]

# sort mag files
# indx = [int(mf.split('.')[-2]) for mf in mag_files]
indx = [int(mf.split('_')[-1].split('.')[0]) for mf in mag_files]
mag_files = [mf for _, mf in sorted(zip(indx, mag_files))]
# get the mesh and magnetisation
mesh = Mesh.from_file(msh_file)

In [7]:
from xmcd_projection.projection import project_structure
p = get_projection_vector(90, 15)

struct = mesh.get_bounding_struct()
struct_projected = project_structure(struct, p)

In [15]:
vis = MeshVisualizer(struct, struct_projected)
vis.show()

In [9]:
1

1

In [10]:
from tqdm import tqdm
import trimesh
from trimesh.ray.ray_triangle import RayMeshIntersector
from xmcd_projection.raytracing import get_piercings_frompt_lengths
from trimesh.ray.ray_triangle import ray_triangle_id
from trimesh import triangles as triangles_mod
from joblib import Parallel, delayed
from time import time
t0 = time()

ray_origins = struct_projected.triangles_center
triangles = mesh.triangles
triangles_normal = triangles_mod.normals(triangles)[0]
tree = triangles_mod.bounds_tree(triangles)

pnew = p[np.newaxis, :]

def ray_id_fun(orig): return ray_triangle_id(
    triangles, orig[np.newaxis, :], pnew, triangles_normal=triangles_normal, tree=tree)
ray_ids_generator = (ray_id_fun(orig) for orig in ray_origins)

# ray_ids_generator = Parallel(n_jobs=5)(delayed(ray_id_fun)(orig) for orig in ray_origins)

print('getting the list')
piercings_list = [
    get_piercings_frompt_lengths(
        ri[2],
        ri[0] // 4)
    for ri in tqdm(ray_ids_generator)]

16it [00:00, 152.40it/s]

getting the list


102208it [12:10, 137.98it/s]

Exception: Wrong number of intersections! Ensure that tetrahedra are valid and that the projection plane does not intersect the structure.

102208it [12:26, 137.98it/s]

In [45]:
raise ValueError

ValueError: 

In [55]:

deep_reload(xmcd_projection)
from xmcd_projection import *
from warnings import warn
from xmcd_projection.raytracing import get_piercings_frompt_lengths

tol = 1e-3

def ray_piercing_fun(orig): return ray_triangle_id(
    triangles, orig[np.newaxis, :], pnew, triangles_normal=triangles_normal, tree=tree)

def get_piercings_item(orig):
    ray_ids, _, locs = ray_piercing_fun(orig)
    try:
        return get_piercings_frompt_lengths(locs, ray_ids // 4)
    except ValueError as e:
        warn(
            str(e) + ' If this happens rarely, it could be a numerical artefact.')
        # running this again, if it crashes a second time, it's not an artefact!
        ray_ids, _, locs = ray_piercing_fun(orig + tol)
        return get_piercings_frompt_lengths(locs, ray_ids // 4)
for i in range(102210, 102220):
#     ri = ray_id_fun(ray_origins[i]+tol)
    get_piercings_item(ray_origins[i])

102216it [46:08, 36.92it/s] 
C:\Users\Dedalo\AppData\Local\Programs\Python\Python36\lib\site-packages\ipykernel_launcher.py:18: UserWarning: Wrong number of intersections! Ensure that tetrahedra are valid and that the projection plane does not intersect the structure. If this happens rarely, it could be a numerical artefact.


In [36]:
np.sort(ri[0])

array([  91300,   91302,  564092,  564095,  971857,  971858, 1072504,
       1072507, 1274697, 1274699, 1377153, 1377155, 1429564, 1429566,
       1543761, 1543763, 1554682, 1554683, 1598737, 1598739, 1620393,
       1620394, 1709497, 1709499, 1746996, 1746998, 1757584, 1757586,
       1827340, 1827342, 1879482, 1879483, 1891620, 1891622, 1899312,
       1899313, 1906993, 1906994, 1943760, 1943761, 1948745, 1948747,
       1957066, 1957067, 1958284, 1958285, 1959572, 1959573, 1979561,
       1979562, 1985252, 1985254])

In [33]:
i = 102216
k = 1952898

In [34]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Create a new plot
fig = plt.figure()
ax = fig.add_subplot((111), projection='3d', proj_type='ortho')

selected_triangles = triangles[(k//4)*4:(k//4)*4+4, :, :]
vts = selected_triangles.reshape(-1, 3)
ax.plot_trisurf(vts[:,0], vts[:,1], vts[:,2])
ax.scatter(vts[:,0], vts[:,1], vts[:,2], 'ko')
ax.set_xlabel('x')

pt0 = ray_origins[i, :]
nx0 = np.cross(pt0, p)
y = np.linspace(vts[:,1].min(), vts[:,1].max(), 100)
# y = np.linspace(points_on_planes[:,1].min(), points_on_planes[:,1].max(), 100)
x = (y*p[0]+nx0[2])/p[1]
z = (y*p[2]-nx0[0])/p[1]

ax.plot(x, y, z, c='red')


In [157]:
pt0 = locations[index_ray == i][4, :]
ax.scatter(pt0[0], pt0[1], pt0[2], c='yellow', s=1000)

In [120]:
locations

array([[111.91509843,  22.08599224,  15.62172815],
       [111.91509843,  22.08599224,  15.62172815],
       [111.91509843,  22.38628166,  15.70219046],
       ...,
       [ -4.39426193, -95.9793848 , 442.39283833],
       [ -4.39426193, -94.09961778, 442.89652038],
       [ -4.39426193, -94.09961778, 442.89652038]])

In [61]:
keep_indices = np.where(diff_tri<=3)
keep_indices = 

(array([0, 2], dtype=int64),)

In [54]:
np.diff(wrong_tri)

array([    1,   488,     1,   651,     2,   326,     2,   110,     2,
        1285,     1, 14211,     1,  4176], dtype=int64)

In [49]:
odd = np.array([len(index_triangle[index_ray == i]) % 2 != 0 for i in range(ray_origins.shape[0])])
np.nonzero(odd)

(array([1095, 1260, 1416, 1509, 1562, 1823, 2169, 2213, 2349, 2406, 2557,
        2911, 3705, 4567, 4727, 4905, 4918, 5072, 5176, 5549], dtype=int64),)

In [ ]:
tetra_wrong = 

In [42]:
np.sort(tri0)

array([    0,     2,     5,     7,    22,    23,    44,    46,    52,
          54,    65,    67,   142,   143,   165,   166,   234,   235,
         325,   326,  1348,  1349,  1376,  1379,  3152,  3153, 29345,
       29346], dtype=int64)

In [34]:
mesh.triangles

array([[[115.07399134,  25.58664678,  16.26597271],
        [116.34854464,  21.08865516,  21.4879623 ],
        [110.80483705,  22.32861196,  14.58337199]],

       [[111.19838654,  22.15640576,  23.50294614],
        [116.34854464,  21.08865516,  21.4879623 ],
        [110.80483705,  22.32861196,  14.58337199]],

       [[111.19838654,  22.15640576,  23.50294614],
        [115.07399134,  25.58664678,  16.26597271],
        [110.80483705,  22.32861196,  14.58337199]],

       ...,

       [[ -2.29675924, -92.49135962, 444.72822183],
        [ -6.06352082, -89.76801016, 447.38076042],
        [  0.84311385, -89.43834269, 444.52670159]],

       [[ -2.29675924, -92.49135962, 444.72822183],
        [ -1.738934  , -88.66054527, 441.8358262 ],
        [  0.84311385, -89.43834269, 444.52670159]],

       [[ -2.29675924, -92.49135962, 444.72822183],
        [ -1.738934  , -88.66054527, 441.8358262 ],
        [ -6.06352082, -89.76801016, 447.38076042]]])